### Запуск PySpark 

In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("test")
         .getOrCreate())

### Подготовка данных

In [17]:
rddUData = spark.sparkContext.textFile("/labs/laba01/ml-100k/u.data")

In [14]:
rddUData.take(5)

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596']

In [18]:
rddUDataSeparated = rddUData.map(lambda x: x.split("\t"))

In [19]:
rddUDataSeparated.take(5)

[['196', '242', '3', '881250949'],
 ['186', '302', '3', '891717742'],
 ['22', '377', '1', '878887116'],
 ['244', '51', '2', '880606923'],
 ['166', '346', '1', '886397596']]

### Подсчет оценок для всех фильмов

#### Отделяю каждую возможную оценку в отдельный rdd:

In [22]:
rddRating1 = rddUDataSeparated.filter(lambda x: x[2] == "1")
rddRating2 = rddUDataSeparated.filter(lambda x: x[2] == "2")
rddRating3 = rddUDataSeparated.filter(lambda x: x[2] == "3")
rddRating4 = rddUDataSeparated.filter(lambda x: x[2] == "4")
rddRating5 = rddUDataSeparated.filter(lambda x: x[2] == "5")

In [21]:
rddRating1.take(5)

[['22', '377', '1', '878887116'],
 ['166', '346', '1', '886397596'],
 ['181', '1081', '1', '878962623'],
 ['276', '796', '1', '874791932'],
 ['20', '288', '1', '879667584']]

#### Определяю количество каждой из оценок:

In [26]:
rddRating1Count = rddRating1.count()
rddRating2Count = rddRating2.count()
rddRating3Count = rddRating3.count()
rddRating4Count = rddRating4.count()
rddRating5Count = rddRating5.count()

#### Проверяю себя:

In [27]:
rddRating1Count + rddRating2Count + rddRating3Count + rddRating4Count + rddRating5Count

100000

## Подсчет оценок для отдельного фильма

#### Отделяю оценки фильма с id 328:

In [28]:
rddUDataMyFilm = rddUDataSeparated.filter(lambda x: x[1] == "328")

In [29]:
rddUDataMyFilm.take(5)

[['166', '328', '5', '886397722'],
 ['119', '328', '4', '876923913'],
 ['38', '328', '4', '892428688'],
 ['229', '328', '1', '891632142'],
 ['111', '328', '4', '891679939']]

#### Отделяю каждую возможную оценку в отдельный rdd:

In [30]:
rddMyFilmRating1 = rddUDataMyFilm.filter(lambda x: x[2] == "1")
rddMyFilmRating2 = rddUDataMyFilm.filter(lambda x: x[2] == "2")
rddMyFilmRating3 = rddUDataMyFilm.filter(lambda x: x[2] == "3")
rddMyFilmRating4 = rddUDataMyFilm.filter(lambda x: x[2] == "4")
rddMyFilmRating5 = rddUDataMyFilm.filter(lambda x: x[2] == "5")

#### Определяю количество каждой из оценок:

In [33]:
rddMyFilmRating1Count = rddMyFilmRating1.count()
rddMyFilmRating2Count = rddMyFilmRating2.count()
rddMyFilmRating3Count = rddMyFilmRating3.count()
rddMyFilmRating4Count = rddMyFilmRating4.count()
rddMyFilmRating5Count = rddMyFilmRating5.count()

In [37]:
spark.stop()

### Сохранение результатов

#### Объединяю все подсчитанные значения в словарь заданного формата:

In [34]:
res = {
           "hist_film": [  
              rddMyFilmRating1Count,
              rddMyFilmRating2Count,
              rddMyFilmRating3Count,
              rddMyFilmRating4Count,
              rddMyFilmRating5Count
           ],
           "hist_all": [  
              rddRating1Count,
              rddRating2Count,
              rddRating3Count,
              rddRating4Count,
              rddRating5Count
           ]
      }

In [35]:
res

{'hist_film': [12, 40, 94, 109, 40],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}

#### Записываю словарь в файл:

In [38]:
import json

with open('lab01.json', 'w') as f:
    json.dump(res, f)